In [45]:
import requests

url = "https://raw.githubusercontent.com/karpathy/ng-video-lecture/master/input.txt"

# Fazendo a requisição GET para obter o conteúdo do arquivo
response = requests.get(url)

# Verificando se a requisição foi bem-sucedida (status code 200)
if response.status_code == 200:
    # Salvar o conteúdo do arquivo
    with open("input.txt", "wb") as file:
        file.write(response.content)
    print("Arquivo baixado com sucesso!")
else:
    print("Erro ao baixar o arquivo. Status code:", response.status_code)


Arquivo baixado com sucesso!


In [46]:
# Explorando os dados

with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [47]:
print("Tamanho do datatset(caracteres): ", len(text))

Tamanho do datatset(caracteres):  1115394


In [48]:
#Printa as primeiros 1000 caracteres
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [49]:
#Pegar o Vocabulario do Texto
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [50]:
print(vocab_size)

65


In [51]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # transforma um string em um  array de inteiros
decode = lambda l: ''.join(itos[i] for i in l) #transforma uma lista de inteiro em uma string
print(encode('Ola Pessoal'))
print(decode(encode('Ola Pessoal')))

[27, 50, 39, 1, 28, 43, 57, 57, 53, 39, 50]
Ola Pessoal


In [52]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
data.shape, data.dtype

(torch.Size([1115394]), torch.int64)

In [53]:
    data[:1000]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57,  1, 47, 57,  1, 41, 

In [54]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [55]:
block_size = 8
train_data[:block_size]

tensor([18, 47, 56, 57, 58,  1, 15, 47])

In [56]:
x = train_data[:block_size] #input
y = train_data[1:block_size+1] #target
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"Quando o input é {context} o target: {target}")

Quando o input é tensor([18]) o target: 47
Quando o input é tensor([18, 47]) o target: 56
Quando o input é tensor([18, 47, 56]) o target: 57
Quando o input é tensor([18, 47, 56, 57]) o target: 58
Quando o input é tensor([18, 47, 56, 57, 58]) o target: 1
Quando o input é tensor([18, 47, 56, 57, 58,  1]) o target: 15
Quando o input é tensor([18, 47, 56, 57, 58,  1, 15]) o target: 47
Quando o input é tensor([18, 47, 56, 57, 58,  1, 15, 47]) o target: 58


In [57]:
torch.manual_seed(1337)
batch_size = 4 # quantas sequencias computadas em paralelo
block_size = 8 # tamanho do contexto 

def get_batch(split):
    # dividindo o dataset em batches de tamanho block_size
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # dimensao do batch
    for t in range(block_size): # dimensao do bloco
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

In [58]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # Cada token lê diretamente os logits para o próximo token a partir de uma tabela de busca
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx e targets são ambos tensores (B, T) de inteiros
        logits = self.token_embedding_table(idx) # (B, T, C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx é um array (B, T) de índices no contexto atual
        for _ in range(max_new_tokens):
            # obter as previsões
            logits, loss = self(idx)
            # focar apenas no último passo de tempo
            logits = logits[:, -1, :] # se torna (B, C)
            # aplicar softmax para obter probabilidades
            probs = F.softmax(logits, dim=-1) # (B, C)
            # amostrar da distribuição
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # anexar o índice amostrado à sequência em execução
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

# Exemplo de uso do modelo
# Suponha que 'vocab_size', 'xb' e 'yb' sejam previamente definidos
# vocab_size = 10000  # Exemplo de tamanho do vocabulário
m = BigramLanguageModel(vocab_size)

# Calcula os logits e a perda
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

# Gera texto usando o modelo
generated_text_idx = m.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()
print(decode(generated_text_idx))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [59]:
# Optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [61]:
batch_size = 32
for steps in range(2000): # Numero de Loops de Treinamento

    # Pega um Batch do treinamento
    xb, yb = get_batch('train')

    # Calcula a Perda
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


3.108039140701294


In [63]:
#Testando para ver se melhorou
generated_text_idx = m.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()
print(decode(generated_text_idx))



Ondzd mpthFInelas s$Ikest3fx' f!Bu!qd,' zenot-iskHSqQYom:wwLthSIoieDmpat


SHWc'Pdq-haratpavM: 'X&y W:k$Whaboth$
pu-oin?KrB-ky;bHAwXNXEXreSck tyIDJU wivVT: fjXUEGUxniOdduFLvfiNIkq-jjHg. A:CESP nXes wN!3f TV;.
DWe:w  nEtxjxLgws pwaRE$z,Ste GG
CEST; rcoBLzmyehogu cgxcV;R?geallvo-dserd&vmrtD&IUBis sxJr d,
UGeqq nlor, vyYN
D&oDNRjban Iy,crmbenzoorqtich .:,llgXie?scUN.
Vesoo'tr Brer&inghotGOUEkPlfkHIF&yZ&
Y,
ObTEDAmTohy firly eSBcormUnsifyZRQ!!'MxqKWV&!ERCJAbj$KonlkK.cumomuOp$VO myhizouAdPhenths!Rj;
